# Analyzing Ford GoBike System

## Investigation Overview
in this analysis i will be looking at: time of most trips taken in terms of time of day;
ii. Day of the week, or month of the year?
iii. How long does the average trip 
take?
iv. Biker type of the bay Area bike share system

# DataSet Overview
This data set includes information about individual rides made in a bike-sharing system covering the greater San Francisco Bay area

In [2]:
# importing packages needed and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

In [4]:
# loading dataset into  pandas dataframe
ford_bike = pd.read_csv('fordgobike.csv')

In [6]:
ford_bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183412 entries, 0 to 183411
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   duration_sec             183412 non-null  int64  
 1   start_time               183412 non-null  object 
 2   end_time                 183412 non-null  object 
 3   start_station_id         183215 non-null  float64
 4   start_station_name       183215 non-null  object 
 5   start_station_latitude   183412 non-null  float64
 6   start_station_longitude  183412 non-null  float64
 7   end_station_id           183215 non-null  float64
 8   end_station_name         183215 non-null  object 
 9   end_station_latitude     183412 non-null  float64
 10  end_station_longitude    183412 non-null  float64
 11  bike_id                  183412 non-null  int64  
 12  user_type                183412 non-null  object 
 13  member_birth_year        175147 non-null  float64
 14  memb

In [7]:
ford_bike.head()

duration_sec                start_time                  end_time  \
0         52185  2019-02-28 17:32:10.1450  2019-03-01 08:01:55.9750   
1         42521  2019-02-28 18:53:21.7890  2019-03-01 06:42:03.0560   
2         61854  2019-02-28 12:13:13.2180  2019-03-01 05:24:08.1460   
3         36490  2019-02-28 17:54:26.0100  2019-03-01 04:02:36.8420   
4          1585  2019-02-28 23:54:18.5490  2019-03-01 00:20:44.0740   

   start_station_id                                start_station_name  \
0              21.0  Montgomery St BART Station (Market St at 2nd St)   
1              23.0                     The Embarcadero at Steuart St   
2              86.0                           Market St at Dolores St   
3             375.0                           Grove St at Masonic Ave   
4               7.0                               Frank H Ogawa Plaza   

   start_station_latitude  start_station_longitude  end_station_id  \
0               37.789625              -122.400811            13.0   
1               37.791464              -122.391034            81.0   
2               37.769305              -122.426826             3.0   
3               37.774836              -122.446546            70.0   
4               37.804562              -122.271738           222.0   

                               end_station_name  end_station_latitude  \
0                Commercial St at Montgomery St             37.794231   
1                            Berry St at 4th St             37.775880   
2  Powell St BART Station (Market St at 4th St)             37.786375   
3                        Central Ave at Fell St             37.773311   
4                         10th Ave at E 15th St             37.792714   

   end_station_longitude  bike_id   user_type  member_birth_year  \
0            -122.402923     4902    Customer             1984.0   
1            -122.393170     2535    Customer                NaN   
2            -122.404904     5905    Customer             1972.0   
3            -122.444293     6638  Subscriber             1989.0   
4            -122.248780     4898  Subscriber             1974.0   

  member_gender bike_share_for_all_trip  
0          Male                      No  
1           NaN                      No  
2          Male                      No  
3         Other                      No  
4          Male                     Yes

In [8]:
ford_bike.isnull().sum()

duration_sec                  0
start_time                    0
end_time                      0
start_station_id            197
start_station_name          197
start_station_latitude        0
start_station_longitude       0
end_station_id              197
end_station_name            197
end_station_latitude          0
end_station_longitude         0
bike_id                       0
user_type                     0
member_birth_year          8265
member_gender              8265
bike_share_for_all_trip       0
dtype: int64

In [11]:
ford_bike.duplicated().sum()

0

# Data Cleaning


In [12]:
#fix 1
# Fixing data types of incorrect datatypes
ford_bike['start_time'] = pd.to_datetime(ford_bike['start_time'])
ford_bike['end_time'] = pd.to_datetime(ford_bike['end_time'])

ford_bike['user_type'] = ford_bike['user_type'].astype('category')
ford_bike['member_gender'] = ford_bike['member_gender'].astype('category')

ford_bike['start_station_id'] = ford_bike['start_station_id'].astype('str')
ford_bike['end_station_id'] = ford_bike['end_station_id'].astype('str')
ford_bike['bike_id'] = ford_bike['bike_id'].astype('str')

ford_bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183412 entries, 0 to 183411
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   duration_sec             183412 non-null  int64         
 1   start_time               183412 non-null  datetime64[ns]
 2   end_time                 183412 non-null  datetime64[ns]
 3   start_station_id         183412 non-null  object        
 4   start_station_name       183215 non-null  object        
 5   start_station_latitude   183412 non-null  float64       
 6   start_station_longitude  183412 non-null  float64       
 7   end_station_id           183412 non-null  object        
 8   end_station_name         183215 non-null  object        
 9   end_station_latitude     183412 non-null  float64       
 10  end_station_longitude    183412 non-null  float64       
 11  bike_id                  183412 non-null  object        
 12  user_type       

In [15]:
# fix 2
# Adding new olumns of trip; duration in minute, trip start hour of the day, trip start day of the week, start month
ford_bike['duration_minute'] = ford_bike['duration_sec']/60

ford_bike['start_date'] = ford_bike.start_time.dt.strftime('%Y-%m-%d')
ford_bike['start_hour_of_day'] = ford_bike.start_time.dt.strftime('%H')
ford_bike['start_day_of_week'] = ford_bike.start_time.dt.strftime('%A')
ford_bike['start_month'] = ford_bike.start_time.dt.strftime('%B')

ford_bike.head()

duration_sec              start_time                end_time  \
0         52185 2019-02-28 17:32:10.145 2019-03-01 08:01:55.975   
1         42521 2019-02-28 18:53:21.789 2019-03-01 06:42:03.056   
2         61854 2019-02-28 12:13:13.218 2019-03-01 05:24:08.146   
3         36490 2019-02-28 17:54:26.010 2019-03-01 04:02:36.842   
4          1585 2019-02-28 23:54:18.549 2019-03-01 00:20:44.074   

  start_station_id                                start_station_name  \
0             21.0  Montgomery St BART Station (Market St at 2nd St)   
1             23.0                     The Embarcadero at Steuart St   
2             86.0                           Market St at Dolores St   
3            375.0                           Grove St at Masonic Ave   
4              7.0                               Frank H Ogawa Plaza   

   start_station_latitude  start_station_longitude end_station_id  \
0               37.789625              -122.400811           13.0   
1               37.791464              -122.391034           81.0   
2               37.769305              -122.426826            3.0   
3               37.774836              -122.446546           70.0   
4               37.804562              -122.271738          222.0   

                               end_station_name  end_station_latitude  ...  \
0                Commercial St at Montgomery St             37.794231  ...   
1                            Berry St at 4th St             37.775880  ...   
2  Powell St BART Station (Market St at 4th St)             37.786375  ...   
3                        Central Ave at Fell St             37.773311  ...   
4                         10th Ave at E 15th St             37.792714  ...   

   member_birth_year member_gender bike_share_for_all_trip  duration_minute  \
0             1984.0          Male                      No       869.750000   
1                NaN           NaN                      No       708.683333   
2             1972.0          Male                      No      1030.900000   
3             1989.0         Other                      No       608.166667   
4             1974.0          Male                     Yes        26.416667   

   start_date start_hourofday  start_dayofweek start_month start_hour_of_day  \
0  2019-02-28              17         Thursday    February                17   
1  2019-02-28              18         Thursday    February                18   
2  2019-02-28              12         Thursday    February                12   
3  2019-02-28              17         Thursday    February                17   
4  2019-02-28              23         Thursday    February                23   

  start_day_of_week  
0          Thursday  
1          Thursday  
2          Thursday  
3          Thursday  
4          Thursday  

[5 rows x 23 columns]